# War card game analysis
Assumptions:
- if there are n players, then there every player gets *(52 mod n)* cards and there are *(52 mod n) * n* cards in game
- non-playing cards are chosen from the lowest ones

e.g. if there are n = 3 players:
    num_of_cards_per_player = 52 mod 3 = 17
    num_of_cards_in_game = 17 * 3 = 51
    deck = [2, 3, 3, 3, 3, 4, ...]

Steps:
1. Defining deck of cards and card values
2. Defining variables for analysis: numer of rounds ...
3. N simulations:
    - shuffling cards and assigning them to n players (1 stack for hand and 1 stack for side cards)
    - N loops:
        1. move - comparison card values
        2. if war, then: 1st move without comparison, 2nd move with comparison (check condition again - recurency)
        3. add cards from the move to winner's side cards stack
        4. if one of the players has all cards, then stop
4. Analysis ...

In [5]:
# libraries
import pandas as pd
import numpy as np

## Simulations

In [8]:
# Defining deck of cards and card values
J = 11
Q = 12
K = 13
A = 14

card_values = list(range(2, 15))
deck = np.array(4 * card_values)
deck.sort()

print(card_values)
print(deck)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[ 2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6  7  7  7  7
  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12 12 12 13 13 13 13
 14 14 14 14]


In [10]:
def make_move(players):
    pass

In [12]:
def war_card_game_simulation(N, players_number):
    # variables for analysis
    rounds_in_each_simulation = []
    players = [ {'hand': np.array(int), 'side': np.array(int)} for _ in range(players_number) ] 
    num_of_cards_in_game = 52 // players_number * players_number
    num_of_cards_per_player = num_of_cards_in_game // players_number
    game_over = False
    
    # simulations
    for i in range(N):
        # shuffling cards 
        shuffled_deck = np.copy(deck)
        shuffled_deck = deck[num_of_cards_in_game:]
        np.random.shuffle(shuffled_deck)

        # assign cards to players' hands
        for player in players:
            player['hand'] = shuffled_deck[:num_of_cards_per_player]
            shuffled_deck = shuffled_deck[num_of_cards_per_player:]
            #print(player['hand'])

        # game until someone wins
        max_iter = 100
        while not game_over:
            make_move(players)
    
            for player in players:
                if len(player['hand']) == num_of_cards_in_game:
                    #print(len(player['hand']))
                    game_over = True
                    break
                    
            max_iter -= 1
            if max_iter <= 0:
                break
    
    return rounds_in_each_simulation

In [14]:
rounds_in_100_simulations = war_card_game_simulation(N = 1, players_number = 2)